# 参数

In [ ]:
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [ ]:
wine = load_wine()
wine.data.shape
wine.target
#如果wine是一张表，应该长这样：
import pandas as pd
pd.concat([pd.DataFrame(wine.data),pd.DataFrame(wine.target)],axis=1)
wine.feature_names
wine.target_names

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(wine.data,wine.target,test_size=0.3)
Xtrain.shape
Xtest.shape

In [ ]:
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf = clf.fit(Xtrain, Ytrain)
score = clf.score(Xtest, Ytest) #返回预测的准确度
score

In [ ]:
feature_name = ['酒精','苹果酸','灰','灰的碱性','镁','总酚','类黄酮','非黄烷类酚类','花青素','颜色强度','色调','od280/od315稀释葡萄酒','脯氨酸']

import graphviz
dot_data = tree.export_graphviz(clf
                               ,feature_names= feature_name#特征的名字
                               ,class_names=["琴酒","雪莉","贝尔摩德"]#指示分成什么类
                               ,filled=True#填充颜色，颜色越浅，不纯度越高
                               ,rounded=True#圆框框
                               )
graph = graphviz.Source(dot_data)
graph

In [ ]:
#特征重要性
clf.feature_importances_

In [ ]:
[*zip(feature_name,clf.feature_importances_)]

In [ ]:
clf = tree.DecisionTreeClassifier(criterion="entropy",random_state=30)#选取随机种子
clf = clf.fit(Xtrain, Ytrain)
score = clf.score(Xtest, Ytest) #返回预测的准确度
score

In [ ]:
clf = tree.DecisionTreeClassifier(criterion="entropy"
                                 ,random_state=30
                                 ,splitter="random"
                                 )
clf = clf.fit(Xtrain, Ytrain)
score = clf.score(Xtest, Ytest)
score

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf
                               ,feature_names= feature_name
                               ,class_names=["琴酒","雪莉","贝尔摩德"]
                               ,filled=True
                               ,rounded=True
                               )  
graph = graphviz.Source(dot_data)
graph

# 预剪枝

In [ ]:
#我们的树对训练集的拟合程度如何？
score_train = clf.score(Xtrain, Ytrain)
score_train

clf = tree.DecisionTreeClassifier(criterion="entropy"
                                 ,random_state=30
                                 ,splitter="random"
                                 ,max_depth=3
                                 ,min_samples_leaf=10#min_samples_leaf限定，一个节点在分枝后的每个子节点都必须包含至少min_samples_leaf个训练样本
                                 ,min_samples_split=10#一个节点必须要包含至少min_samples_split个训练样本
                                 )
clf = clf.fit(Xtrain, Ytrain)
dot_data = tree.export_graphviz(clf
                               ,feature_names= feature_name
                               ,class_names=["琴酒","雪莉","贝尔摩德"]
                               ,filled=True
                               ,rounded=True
                               )  
graph = graphviz.Source(dot_data)
graph

In [ ]:
clf.score(Xtrain,Ytrain)
clf.score(Xtest,Ytest)

In [ ]:
import matplotlib.pyplot as plt
test = []
for i in range(10):
    clf = tree.DecisionTreeClassifier(max_depth=i+1
                                     ,criterion="entropy"
                                     ,random_state=30
                                     ,splitter="random"
                                     )
    clf = clf.fit(Xtrain, Ytrain)
    score = clf.score(Xtest, Ytest)
    test.append(score)
plt.plot(range(1,11),test,color="red",label="max_depth")
plt.legend()
plt.show()

# 重要属性和接口

In [ ]:
#apply返回每个测试样本所在的叶子节点的索引
clf.apply(Xtest)

In [ ]:
#predict返回每个测试样本的分类/回归结果
clf.predict(Xtest)

# 案例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier

#make_classification库生成随机的二分型数据
X, y = make_classification(n_samples=100, #生成100个样本
                           n_features=2,  #包含2个特征，即生成二维数据
                           n_redundant=0, #添加冗余特征0个
                           n_informative=2, #包含信息的特征是2个
                           random_state=1,  #随机模式1
                           n_clusters_per_class=1 #每个簇内包含的标签类别有1个
                         )
#在这里可以查看一下X和y，其中X是100行带有两个2特征的数据，y是二分类标签
#也可以画出散点图来观察一下X中特征的分布
plt.scatter(X[:,0],X[:,1])

In [ ]:
#从图上可以看出，生成的二分型数据的两个簇离彼此很远，这样不利于我们测试分类器的效果，因此我们使用np生成随机数组，通过让已经生成的二分型数据点加减0~1之间的随机数，使数据分布变得更散更稀疏
#注意，这个过程只能够运行一次，因为多次运行之后X会变得非常稀疏，两个簇的数据会混合在一起，分类器的效应会继续下降
rng = np.random.RandomState(2) #生成一种随机模式
X += 2 * rng.uniform(size=X.shape) #加减0~1之间的随机数
linearly_separable = (X, y) #生成了新的X，依然可以画散点图来观察一下特征的分布
plt.scatter(X[:,0],X[:,1])
#用make_moons创建月亮型数据，make_circles创建环形数据，并将三组数据打包起来放在列表datasets中
datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable]

In [ ]:
#创建画布，宽高比为6*9
figure = plt.figure(figsize=(6, 9))
#设置用来安排图像显示位置的全局变量i
i = 1
#开始迭代数据，对datasets中的数据进行for循环
for ds_index, ds in enumerate(datasets):
    
    #对X中的数据进行标准化处理，然后分训练集和测试集
    X, y = ds
    X = StandardScaler().fit_transform(X) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)
    
    #找出数据集中两个特征的最大值和最小值，让最大值+0.5，最小值-0.5，创造一个比两个特征的区间本身更大一点的区间
    x1_min, x1_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    x2_min, x2_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    
    #用特征向量生成网格数据，网格数据，其实就相当于坐标轴上无数个点
    #函数np.arange在给定的两个数之间返回均匀间隔的值，0.2为步长
    #函数meshgrid用以生成网格数据，能够将两个一维数组生成两个二维矩阵。
    #如果第一个数组是narray，维度是n，第二个参数是marray，维度是m。那么生成的第一个二维数组是以narray为行，m行的矩阵，而第二个二维数组是以marray的转置为列，n列的矩阵
    #生成的网格数据，是用来绘制决策边界的，因为绘制决策边界的函数contourf要求输入的两个特征都必须是二维的
    array1,array2 = np.meshgrid(np.arange(x1_min, x1_max, 0.2),
                         np.arange(x2_min, x2_max, 0.2))
    #接下来生成彩色画布
    #用ListedColormap为画布创建颜色，#FF0000正红，#0000FF正蓝
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
    #在画布上加上一个子图，数据为len(datasets)行，2列，放在位置i上
    ax = plt.subplot(len(datasets), 2, i)
 
    #到这里为止，已经生成了0~1之间的坐标系3个了，接下来为我们的坐标系放上标题
    #我们有三个坐标系，但我们只需要在第一个坐标系上有标题，因此设定if ds_index==0这个条件
    if ds_index == 0:
        ax.set_title("Input data")
    
    #将数据集的分布放到我们的坐标系上
    #先放训练集
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, 
               cmap=cm_bright,edgecolors='k')
    #放测试集
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, 
               cmap=cm_bright, alpha=0.6,edgecolors='k')
    
     #为图设置坐标轴的最大值和最小值，并设定没有坐标轴
    ax.set_xlim(array1.min(), array1.max())
    ax.set_ylim(array2.min(), array2.max())
    ax.set_xticks(())
    ax.set_yticks(())
    
    #每次循环之后，改变i的取值让图每次位列不同的位置
    i += 1
    
    #至此为止，数据集本身的图像已经布置完毕，运行以上的代码，可以看见三个已经处理好的数据集

In [ ]:
#迭代决策树，首先用subplot增加子图，subplot(行，列，索引)这样的结构，并使用索引i定义图的位置
#在这里，len(datasets)其实就是3，2是两列
#在函数最开始，我们定义了i=1，并且在上边建立数据集的图像的时候，已经让i+1,所以i在每次循环中的取值是2，4，6
ax = plt.subplot(len(datasets),2,i)

#决策树的建模过程：实例化 → fit训练 → score接口得到预测的准确率
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

#绘制决策边界，为此，我们将为网格中的每个点指定一种颜色[x1_min，x1_max] x [x2_min，x2_max]
#分类树的接口，predict_proba，返回每一个输入的数据点所对应的标签类概率
#类概率是数据点所在的叶节点中相同类的样本数量/叶节点中的样本总数量
#由于决策树在训练的时候导入的训练集X_train里面包含两个特征，所以我们在计算类概率的时候，也必须导入结构相同的数组，即是说，必须有两个特征
#ravel()能够将一个多维数组转换成一维数组
#np.c_是能够将两个数组组合起来的函数
#在这里，我们先将两个网格数据降维降维成一维数组，再将两个数组链接变成含有两个特征的数据，再带入决策树模型，生成的Z包含数据的索引和每个样本点对应的类概率，再切片，且出类概率
Z = clf.predict_proba(np.c_[array1.ravel(),array2.ravel()])[:, 1]

#np.c_[np.array([1,2,3]), np.array([4,5,6])]

#将返回的类概率作为数据，放到contourf里面绘制去绘制轮廓
Z = Z.reshape(array1.shape)
ax.contourf(array1, array2, Z, cmap=cm, alpha=.8)

#将数据集的分布放到我们的坐标系上
# 将训练集放到图中去
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
           edgecolors='k')
# 将测试集放到图中去
ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
           edgecolors='k', alpha=0.6)

#为图设置坐标轴的最大值和最小值
ax.set_xlim(array1.min(), array1.max())
ax.set_ylim(array2.min(), array2.max())
#设定坐标轴不显示标尺也不显示数字
ax.set_xticks(())
ax.set_yticks(())

#我们有三个坐标系，但我们只需要在第一个坐标系上有标题，因此设定if ds_index==0这个条件
if ds_index == 0:
    ax.set_title("Decision Tree")

#写在右下角的数字    
ax.text(array1.max() - .3, array2.min() + .3, ('{:.1f}%'.format(score*100)),
        size=15, horizontalalignment='right')

#让i继续加一
i += 1
plt.tight_layout()
plt.show()

# 回归树拟合正弦函数

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [ ]:
# 创建一个随机数生成器，设置种子为1
rng = np.random.RandomState(1)

# 生成一个80x1的随机数组，每个值在[0, 5)范围内，并按列排序
X = np.sort(5 * rng.rand(80, 1), axis=0)

# 计算y值，y是X的正弦值，并将其展平为一维数组
y = np.sin(X).ravel()
plt.figure()
plt.scatter(X,y,s=20,edgecolor="black",c="darkorange",label="data")#svatter画散点图

# 每隔5个数据点，在y值上添加噪声
y[::5] += 3 * (0.5 - rng.rand(16))#步长为5

plt.figure()
plt.scatter(X,y,s=20,edgecolor="black",c="darkorange",label="data")#svatter画散点图

# 测试ravel()的用法
print("随机数组：", np.random.random((2, 1)))
print("展平后的数组：", np.random.random((2, 1)).ravel())
print("展平后数组的形状：", np.random.random((2, 1)).ravel().shape)

regr_1=DecisionTreeRegressor(max_depth=2)
regr_1.fit(X,y)
regr_2=DecisionTreeRegressor(max_depth=5)
regr_2.fit(X,y)

In [ ]:
X_test=np.arange(0,5,0.01)[:,np.newaxis]#开始点、结束点、步长,切片增维
#print(X_test)
y_1=regr_1.predict(X_test)
y_2=regr_2.predict(X_test)

In [ ]:
plt.figure()
plt.scatter(X,y,s=20,edgecolor="black",c="darkorange",label="data")
plt.plot(X_test,y_1,color="cornflowerblue",label="max_depth=2",linewidth=2)
plt.plot(X_test,y_2,color="yellowgreen",label="max_depth=5",linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

# 泰坦尼克幸存者预测

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

#处理训练集
data=pd.read_csv("./train.csv")
#print(data.info)
#print(data.head(10))
#筛选特征
data.drop(["Cabin","Name","Ticket"],inplace=True,axis=1)#删除列
#print(data.head(10))
#处理缺失值
data["Age"]=data["Age"].fillna(data["Age"].mean())#使用均值填充缺失值
data=data.dropna()#删除有缺失值的行

labels=data["Embarked"].unique().tolist()
data["Embarked"]=data["Embarked"].apply(lambda x:labels.index(x))#将登船状态转换成数字

#data["Sex"]=(data["Sex"]=="male")
data["Sex"]=(data["Sex"]=="male").astype("int")#将布尔值转换为其它数据类型

#data.head(10)

# 取出所有行，排除 "Survived" 列
Xtrain = data.loc[:, data.columns != "Survived"]
Ytrain = data.loc[:, data.columns == "Survived"]  # 取出 "Survived" 列作为目标变量

#print("x:\n", x)
#print("y:\n", y)

#处理测试集
data=pd.read_csv("./test.csv")
#print(data.info)
#print(data.head(10))
#筛选特征
data.drop(["Cabin","Name","Ticket"],inplace=True,axis=1)#删除列
#处理缺失值
data["Age"]=data["Age"].fillna(data["Age"].mean())#使用均值填充缺失值
data=data.dropna()#删除有缺失值的行
labels=data["Embarked"].unique().tolist()
data["Embarked"]=data["Embarked"].apply(lambda x:labels.index(x))#将登船状态转换成数字
#data["Sex"]=(data["Sex"]=="male")
data["Sex"]=(data["Sex"]=="male").astype("int")#将布尔值转换为其它数据类型
# 取出所有行，排除 "Survived" 列
Xtest = data.loc[:, data.columns != "Survived"]
Ytest = data.loc[:, data.columns == "Survived"]  # 取出 "Survived" 列作为目标变量

#print("x:\n", x)
#print("y:\n", y)

#print(Xtrain.shape,Ytrain.shape,Xtest.shape,Ytest.shape)

#print(Xtrain,Ytrain,Xtest,Ytest)

for i in [Xtrain,Xtest,Ytrain,Ytest]:#前面取的是乱序的，重新排一下（纠正索引）
    i.index=range(i.shape[0])
#print(Xtrain,Ytrain,Xtest,Ytest)

clf=DecisionTreeClassifier(random_state=25)
clf=clf.fit(Xtrain,Ytrain)
score=clf.score(Xtest,Ytest)

# kimi泰坦尼克

In [88]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 处理训练集
train_data = pd.read_csv("./train.csv")
# 筛选特征
train_data.drop(["Cabin", "Name", "Ticket"], inplace=True, axis=1)  # 删除列
# 处理缺失值
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].mean())  # 使用均值填充缺失值
train_data = train_data.dropna()  # 删除有缺失值的行
# 将分类变量转换为数值
labels = train_data["Embarked"].unique().tolist()
train_data["Embarked"] = train_data["Embarked"].apply(lambda x: labels.index(x))  # 将登船状态转换成数字
train_data["Sex"] = (train_data["Sex"] == "male").astype("int")  # 将布尔值转换为整数

# 取出所有行，排除 "Survived" 列
Xtrain = train_data.loc[:, train_data.columns != "Survived"]
Ytrain = train_data.loc[:, train_data.columns == "Survived"]  # 取出 "Survived" 列作为目标变量

# 处理测试集
test_data = pd.read_csv("./test.csv")
# 筛选特征
test_data.drop(["Cabin", "Name", "Ticket"], inplace=True, axis=1)  # 删除列
# 处理缺失值
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].mean())  # 使用均值填充缺失值
test_data = test_data.dropna()  # 删除有缺失值的行
# 将分类变量转换为数值
labels = test_data["Embarked"].unique().tolist()
test_data["Embarked"] = test_data["Embarked"].apply(lambda x: labels.index(x))  # 将登船状态转换成数字
test_data["Sex"] = (test_data["Sex"] == "male").astype("int")  # 将布尔值转换为整数

# 重置索引
Xtrain.reset_index(drop=True, inplace=True)
Ytrain.reset_index(drop=True, inplace=True)

# 注意：测试集没有目标变量，因此不需要 Ytest
Xtest = test_data

# 训练决策树模型
clf = DecisionTreeClassifier(random_state=25)
clf.fit(Xtrain, Ytrain)

# 预测测试集
predictions = clf.predict(Xtest)

# 如果需要保存预测结果
submission = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Survived": predictions})
submission.to_csv("submission.csv", index=False)

# 如果需要使用 GridSearchCV 优化超参数
param_grid = {"max_depth": [3, 5, 7, 10], "min_samples_split": [2, 5, 10]}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=25), param_grid, cv=5)
grid_search.fit(Xtrain, Ytrain)
print("最佳参数：", grid_search.best_params_)
print("最佳分数：", grid_search.best_score_)

最佳参数： {'max_depth': 7, 'min_samples_split': 5}
最佳分数： 0.7942042785501174
